# Wrapping a basic library

We here aim at presenting the interactive wrapping workflow.
For the sake of simplicity, we consider a basic example of a *C++* library.

First, we need:

* to detect if the operating system (OS) is a Windows OS or a Unix OS.

In [1]:
import platform
is_windows = any(platform.win32_ver())

* to detect the version of *Python* installed and to save it in the `PYTHON_VERSION` environment variable.

In [2]:
import os
import sys
os.environ['PYTHON_VERSION'] = str(sys.version_info.major) + '.' + str(sys.version_info.minor)

* to import **AutoWIG**.

In [3]:
import autowig

Then, we need to install and compile the *C++* library.
To do so, we use available **Conda** recipes.

In [4]:
!conda remove -y libbasic
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\libbasic -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/libbasic -c statiskit
!conda install -y libbasic --use-local -c statiskit

Fetching package metadata ...........
Solving package specifications: .

PackageNotFoundError: Package(s) is missing from the environment:
            [u'libbasic
            

Attempting to finalize metadata for libbasic
INFO:conda_build.metadata:Attempting to finalize metadata for libbasic
BUILD START: [u'libbasic-1.0.0-hd44bf51_0.tar.bz2']

The following NEW packages will be INSTALLED:

    binutils_impl_linux-64: 2.28.1-h04c84fa_2             
    binutils_linux-64:      7.2.0-25                      
    ca-certificates:        2017.08.26-h1d4fec5_0         
    certifi:                2017.11.5-py27h71e7faf_0      
    gcc_impl_linux-64:      7.2.0-hc5ce805_2              
    gcc_linux-64:           7.2.0-25                      
    gfortran_impl_linux-64: 7.2.0-hb3c8cce_2              
    gfortran_linux-64:      7.2.0-25                      
    gxx_impl_linux-64:      7.2.0-hd3faf3d_2              
    gxx_linux-64:           7.2.0-25                      
    libedit:     

INFO: activate-gfortran_linux-64.sh made the following environmental changes:
+DEBUG_FFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libbasic_1515579352787/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho/include -fdebug-prefix-map=${SRC_DIR}=/usr/local/src/conda/${PKG_NAME}-${PKG_VERSION} -fdebug-prefix-map=${PREFIX}=/usr/local/src/conda-prefix -fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-all -fno-plt -Og -g -Wall -Wextra -fcheck=all -fbacktrace -fimplicit-none -fvar-tracking-assignments -pipe
+DEBUG_FORTRANFLAGS=-fopenmp -march=nocona -mtune=haswell -ftree-vectorize -fPIC -fstack-protector-strong -fno-plt -O2 -pipe -I/home/pfernique/.miniconda2/conda-bld/libbasic_1515579352787/_h_env_placehold_placehold_p

# Automatic uploading is disabled
# If you want to upload package(s) to anaconda.org later, type:

anaconda upload /home/pfernique/.miniconda2/conda-bld/linux-64/libbasic-1.0.0-hd44bf51_0.tar.bz2

# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes

anaconda_upload is not set.  Not uploading wheels: []



####################################################################################
Source and build intermediates have been left in /home/pfernique/.miniconda2/conda-bld.
There are currently 34 accumulated.
To remove them, you can run the ```conda build purge``` command
Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /home/pfernique/.miniconda2/envs/fp17:

The following NEW packages will be INSTALLED:

    libbasic: 1.0.0-hd44bf51_0 local



Once these preliminaries are done, we can proceed to the actual generation of wrappers for this *C++* library.
For this, we create an empty Abstract Semantic Graph (ASG).

In [5]:
asg = autowig.AbstractSemanticGraph()

We then parse the headers of this *C++* library with relevant compilation flags.

In [6]:
%%time

try:
    from path import path as Path
except:
    from path import Path
prefix = Path(sys.prefix).abspath()
if is_windows:
    headers = [prefix/'Library'/'include'/'basic'/'overload.h',
               prefix/'Library'/'include'/'basic'/'binomial.h']
else:
    headers = [prefix/'include'/'basic'/'overload.h',
               prefix/'include'/'basic'/'binomial.h']
    
flags = ['-x', 'c++', '-std=c++11']
if is_windows:
    flags.append('-I' + str((prefix/'Library'/'include').abspath()))
else:
    flags.append('-I' + str((prefix/'include').abspath()))
    
asg = autowig.parser(asg, headers,
                          flags,
                          silent = True)

CPU times: user 700 ms, sys: 60 ms, total: 760 ms
Wall time: 1.83 s


Since most of **AutoWIG** guidelines are respected in this *C++* library, the `default` `controller` implementation is suitable.

In [7]:
%%time

asg = autowig.controller(asg)

CPU times: user 236 ms, sys: 8 ms, total: 244 ms
Wall time: 239 ms


In order to wrap the library we need to select the `boost_python_internal` `generator` implementation.

In [8]:
%%time

autowig.generator.plugin = 'boost_python_internal'  
wrappers = autowig.generator(asg,
                             module = Path('.')/'..'/'..'/'src'/'py'/'_basic.cpp',
                             decorator = Path('.')/'..'/'..'/'src'/'py'/'basic'/'_basic.py',
                             prefix = 'wrapper_')

CPU times: user 48 ms, sys: 8 ms, total: 56 ms
Wall time: 52 ms


The wrappers are only generated in-memory.
It is therefore needed to write them on the disk to complete the process.

In [9]:
%%time

wrappers.write()

CPU times: user 436 ms, sys: 616 ms, total: 1.05 s
Wall time: 3.04 s


Here is the list of the generated wrappers (untracked files).

In [10]:
!git status

On branch master
Your branch is up-to-date with 'origin/master'.
Untracked files:
  (use "git add <file>..." to include in what will be committed)

	../../src/py/_basic.cpp
	../../src/py/_basic.h
	../../src/py/basic/_basic.py
	../../src/py/wrapper_0f744e8d056f5d469a887c7c78eaf8fe.cpp
	../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
	../../src/py/wrapper_92d464d1243554009adc48a065a5be3e.cpp
	../../src/py/wrapper_f926cb231a7f5da09f313cd361ff94c7.cpp

nothing added to commit but untracked files present (use "git add" to track)


And here, we present the wrappers generated for the `BinomialDistribution` class.

In [11]:
if is_windows:
    !pygmentize ..\..\src\py\wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp
else:
    !pygmentize ../../src/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp

#include "_basic.h"



namespace autowig
{

}

#if defined(_MSC_VER)
    #if (_MSC_VER == 1900)
namespace boost
{
    template <> class ::BinomialDistribution const volatile * get_pointer<class ::BinomialDistribution const volatile >(class ::BinomialDistribution const volatile *c) { return c; }
}
    #endif
#endif



void wrapper_4046a8421fe9587c9dfbc97778162c7d()
{

    double  (::BinomialDistribution::*method_pointer_3a3ff64f25e358a6a10b1cd3b3425b82)(unsigned int const) const = &::BinomialDistribution::pmf;
    double  (::BinomialDistribution::*method_pointer_3c97a500c9575c259d5cbdd76120ff4f)() const = &::BinomialDistribution::get_pi;
    void  (::BinomialDistribution::*method_pointer_d364a0529e33516f8ecbb7dcedd60aa0)(double const) = &::BinomialDistribution::set_pi;
    boost::python::class_< class ::BinomialDistribution, autowig::Held< class ::BinomialDistribution >::Type > class_4046a8421fe9587c9dfbc97778162c7d("BinomialDistribution", "", boost::python::no_init);
    class_4046a842

Once the wrappers are written on disk, we need to compile and install the *Python* bindings.
To do so, we use available **Conda** recipes.

In [12]:
!conda remove -y python-basic
if is_windows:
    !conda build --python=%PYTHON_VERSION% ..\..\bin\conda\python-basic -c statiskit
else:
    !conda build --python=$PYTHON_VERSION ../../bin/conda/python-basic -c statiskit
!conda install -y python-basic --use-local -c statiskit

Fetching package metadata ...........
Solving package specifications: .

PackageNotFoundError: Package(s) is missing from the environment:
            [u'python-basic
            

Attempting to finalize metadata for python-basic
INFO:conda_build.metadata:Attempting to finalize metadata for python-basic
BUILD START: [u'python-basic-1.0.0-py27hc781fe7_0.tar.bz2']

The following NEW packages will be INSTALLED:

    alabaster:                0.7.10-py27he5a193a_0         
    asn1crypto:               0.23.0-py27h7b0c810_0         
    babel:                    2.5.0-py27h20693cd_0          
    binutils_impl_linux-64:   2.28.1-h04c84fa_2             
    binutils_linux-64:        7.2.0-25                      
    bzip2:                    1.0.6-h6d464ef_2              
    ca-certificates:          2017.08.26-h1d4fec5_0         
    certifi:                  2017.11.5-py27h71e7faf_0      
    cffi:                     1.11.2-py27ha7929c6_0         
    chardet:                  3.0.4-py

INFO: activate-binutils_linux-64.sh made the following environmental changes:
+ADDR2LINE=/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/bin/x86_64-conda_cos6-linux-gnu-addr2line
+AR=/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/bin/x86_64-conda_cos6-linux-gnu-ar
+AS=/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/bin/x86_64-conda_cos6-linux-gnu-as
+CXXFILT=/home/pferniqu

scons: Reading SConscript files ...
scons: done reading SConscript files.
scons: Building targets ...
Install file: "build/cpp/overload.h" as "/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/basic/overload.h"
Install file: "build/cpp/binomial.h" as "/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/basic/binomial.h"
Creating 'build/py/response_file.rsp'
Install file: "build/cpp/base.h" as "/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_place

In file included from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/object/function_handle.hpp:10:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/converter/arg_to_python.hpp:19,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/call.hpp:15,
                 from /ho

/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/bin/x86_64-conda_cos6-linux-gnu-g++ -o build/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.os -c -std=c++11 -fvisibility=hidden -Wl,--no-undefined -fdiagnostics-color=always -fPIC -DBOOST_PYTHON_DYNAMIC_LIB -DBOOST_ALL_NO_LIB -I/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include -I/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_

...

In file included from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python.hpp:41:0,
                 from build/py/_basic.h:4,
                 from build/py/wrapper_4046a8421fe9587c9dfbc97778162c7d.cpp:1:
/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/make_constructor.hpp: In member function 'void boost::python::detail::install_holder<T>::dispatch(U*, mpl_::true_) const':
/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placeho

In file included from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/function/detail/maybe_include.hpp:23:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/function/function2.hpp:11,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/object/function_object.hpp:8,
                 fro

In file included from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/function/detail/maybe_include.hpp:23:0,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/function/function2.hpp:11,
                 from /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/include/boost/python/object/function_object.hpp:8,
                 fro

/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/bin/x86_64-conda_cos6-linux-gnu-gcc -o /home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib/python2.7/site-packages/basic/__basic.so @/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/work/build/py/response_file.rsp -shared -L/home/pfernique/.miniconda2/conda-bld/python-basic_1515579380129/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_pla/lib -lboost_python -lpython2.7 -lbasic
s

# Automatic uploading is disabled
# If you want to upload package(s) to anaconda.org later, type:

anaconda upload /home/pfernique/.miniconda2/conda-bld/linux-64/python-basic-1.0.0-py27hc781fe7_0.tar.bz2

# To have conda build upload to anaconda.org automatically, use
# $ conda config --set anaconda_upload yes

anaconda_upload is not set.  Not uploading wheels: []



####################################################################################
Source and build intermediates have been left in /home/pfernique/.miniconda2/conda-bld.
There are currently 34 accumulated.
To remove them, you can run the ```conda build purge``` command
Fetching package metadata ...............
Solving package specifications: .

Package plan for installation in environment /home/pfernique/.miniconda2/envs/fp17:

The following NEW packages will be INSTALLED:

    libboost_core-dev:   1.61.0-0             local
    libboost_python-dev: 1.61.0-py27_0        local
    python-basic:        1.0.0-py27hc781fe7_

Finally, we can hereafter use the *C++* library in the *Python* interpreter.

In [13]:
import basic
binomial = basic.BinomialDistribution(1, .5)
binomial

In [14]:
binomial.pmf(0)

0.5

In [15]:
binomial.pmf(1)

0.5

In [16]:
binomial.n = 0
binomial

In [17]:
binomial.pmf(0)

1.0

In [18]:
try:
    binomial.set_pi(1.1)
except basic.ProbabilityError as error:
    print(error)
else:
    raise Exception('A `basic.ProbabilityError` should have been raised')

a probability must be in the interval [0,1]


Here is a report concerning objects wrapped using this notebook.

In [19]:
import fp17
fp17.report(asg)

Headers: 3 (170 SLOC)
Variables: 1 (1 wrapped)
Functions: 15 (10 wrapped)
Classes: 4 (4 wrapped)
